In [1]:
pip install -U keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 4.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
import tensorflow as tf 
from tensorflow.keras.layers import Dense
import keras_tuner as kf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/aishwaryamate/Datasets/main/Churn_Modelling.csv')

In [5]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [6]:
df.drop(columns = ['RowNumber'	,'CustomerId',	'Surname'], inplace=True)

In [7]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [8]:
df = pd.get_dummies(data = df,columns=['Geography','Gender'])
df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,1,0,0,0,1
9996,516,35,10,57369.61,1,1,1,101699.77,0,1,0,0,0,1
9997,709,36,7,0.00,1,0,1,42085.58,1,1,0,0,1,0
9998,772,42,3,75075.31,2,1,0,92888.52,1,0,1,0,0,1


In [9]:
x = df.drop(columns = ['Exited'])
y = df['Exited']

In [11]:
sc = StandardScaler()

x = sc.fit_transform(x)

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size = 0.2, random_state=42)

In [14]:
ann = Sequential()

ann.add(Dense(units = 10, activation = 'relu'))
ann.add(Dense(units = 1, activation = 'sigmoid'))

ann.compile(optimizer = 'adadelta', loss = 'binary_crossentropy', metrics ='accuracy')

ann.fit(xtrain,ytrain,epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.8353 - accuracy: 0.5567
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.8339 - accuracy: 0.5579
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.8325 - accuracy: 0.5586
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.8311 - accuracy: 0.5587
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.8297 - accuracy: 0.5601
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.8283 - accuracy: 0.5609
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 0.8269 - accuracy: 0.5614
Epoch 8/100
250/250 [==============================] - 1s 2ms/step - loss: 0.8255 - accuracy: 0.5616
Epoch 9/100
250/250 [==============================] - 1s 2ms/step - loss: 0.8241 - accuracy: 0.5621
Epoch 10/100
250/250 [==============================] - 1s 2ms/step - loss: 0.8228 - accura

In [15]:
ypred = ann.predict(xtest)
ypred = ypred > 0.5

63/63 [==============================] - 0s 1ms/step


In [16]:
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.80      0.72      0.76      1607
           1       0.19      0.26      0.22       393

    accuracy                           0.63      2000
   macro avg       0.49      0.49      0.49      2000
weighted avg       0.68      0.63      0.65      2000



In [ ]:
#Hyperparameter tuning

In [23]:
def hyper(hp):
    model = Sequential()
    counter = 0
    for i in range(hp.Int('num_layers', min_value = 1,max_value=10)):
        if counter ==0:
            model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
                            activation = hp.Choice('activation'+str(i),values = ['relu','tanh','sigmoid']),
                            input_dim = 13))
            counter = counter + 1  
        else:
          model.add(Dense(
              hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
              activation = hp.Choice('activation'+str(i),values = ['relu','tanh','sigmoid'])))
        
    model.add(Dense(units = 1,activation='sigmoid'))
    model.compile(optimizer = hp.Choice('optimizer',values = ['adam','rmsprop']),
                loss = 'binary_crossentropy',metrics = 'accuracy')
    return model

In [26]:
tuner = kf.RandomSearch(hyper,
                        objective = 'val_accuracy',
                        max_trials = 3,
                        directory = 'project',
                        project_name = 'File1')

In [27]:
tuner.search(xtrain,ytrain,epochs = 5, validation_data = (xtest,ytest))

Trial 3 Complete [00h 00m 07s]
val_accuracy: 0.8034999966621399

Best val_accuracy So Far: 0.859000027179718
Total elapsed time: 00h 00m 19s


In [28]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 5,
 'units0': 72,
 'activation0': 'relu',
 'optimizer': 'adam',
 'units1': 8,
 'activation1': 'relu',
 'units2': 8,
 'activation2': 'relu',
 'units3': 8,
 'activation3': 'relu',
 'units4': 8,
 'activation4': 'relu'}

In [29]:
model = tuner.get_best_models(num_models=1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 72)                1008      
                                                                 
 dense_1 (Dense)             (None, 8)                 584       
                                                                 
 dense_2 (Dense)             (None, 8)                 72        
                                                                 
 dense_3 (Dense)             (None, 8)                 72        
                                                                 
 dense_4 (Dense)             (None, 8)                 72        
                                                                 
 dense_5 (Dense)             (None, 1)                 9         
                                                                 
Total params: 1,817
Trainable params: 1,817
Non-trainabl

In [30]:
model.fit(xtrain,ytrain,epochs = 100)

Epoch 1/100
250/250 [==============================] - 2s 2ms/step - loss: 0.3393 - accuracy: 0.8604
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3349 - accuracy: 0.8622
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3309 - accuracy: 0.8658
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3263 - accuracy: 0.8694
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3236 - accuracy: 0.8690
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3220 - accuracy: 0.8671
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3191 - accuracy: 0.8673
Epoch 8/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3169 - accuracy: 0.8709
Epoch 9/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3149 - accuracy: 0.8715
Epoch 10/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3142 - accura